In [31]:
import os
from typing import Optional, Union
import numpy as np
import pandas as pd
import seaborn as sns
import ast
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
def clean_df(df_raw) -> pd.DataFrame:
    df = df_raw.copy()
    df['main_category'] = df['category'].apply(
        lambda x: ast.literal_eval(x)['slug'].split('/')[0]
    )

    df['successful'] = pd.Series(df['goal'] <= df['converted_pledged_amount'],dtype=np.int8)
    df.drop(columns=['goal','converted_pledged_amount'],inplace=True)

    unix_cols = ['launched_at','deadline','state_changed_at']
    for col in unix_cols :
        df[col] = pd.to_datetime(df[col],origin='unix', format='%Y%m%d')

    scrape_date = datetime.datetime(2020,9,17)
    df = df[df['deadline'] < scrape_date]
    df['deadline'] = (df['deadline'] - df['launched_at']).dt.days

    drop_cols = ['blurb','country','creator','currency_symbol','current_currency',
                'country_displayable_name','creator','location','photo',
                'profile','slug','source_url','urls','is_backing','is_starred',
                'static_usd_rate','usd_pledged','usd_type','category','friends',
                 'permissions','created_at', 'currency_trailing_code']
    to_datetime_cols = []
    to_ord_cols = []
    to_one_hot_cols = ['currency']

    df.drop(columns=drop_cols,inplace=True)
    
    for col in to_datetime_cols :
        df[col] = pd.to_datetime(df[col],origin='unix', format='%Y%m%d')
    
    for col in to_one_hot_cols:
        one_hot = pd.get_dummies(df[col],prefix=col,drop_first=True)
        df = df.join(one_hot)
    df.drop(columns=to_one_hot_cols,inplace=True)

    mid = df['name']
    df.drop(labels=['name'], axis=1, inplace = True)
    df.insert(0, 'name', mid)

    return df

In [75]:
def join_csv(search_path, max_tables:Optional[int]=None) -> pd.DataFrame:
    out_table = pd.DataFrame()
    if max_tables is None: max_tables = np.inf
    num_tables = 0
    for root, dirs, files in os.walk(search_path, topdown=True):
        dirs.sort(reverse=True)
        for name in files:
            if num_tables > max_tables : 
                return out_table.drop_duplicates(subset='name', keep='first')
            table_path = os.path.join(root, name)
            current_table = pd.read_csv(table_path)
            current_table = clean_df(current_table)
            if out_table.shape[0] == 0: 
                out_table = current_table
                continue
            
            out_table = out_table.merge(current_table, how='inner')
            
            num_tables += 1
    return out_table.drop_duplicates(subset='name', keep='first')

In [76]:
df = join_csv('E:\KickStarter\input', max_tables=4)

ValueError: unconverted data remains: 5385

In [ ]:
df.isna().sum().sum()

In [77]:
df.head()

,name,backers_count,deadline,disable_communication,fx_rate,id,is_starrable,launched_at,pledged,spotlight,...,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD
0,"MY BROKEN CAMERA - ""SELF (i.e.) PORTRAIT""- Pho...",64,0,False,1.000000,1989079754,False,1970-01-01 00:00:01.450499598,4185.0,True,...,0,0,0,0,0,0,0,0,0,1
1,"Pole Diversity: A Showcase of Pole, Aerial, an...",6,0,False,1.000000,233452533,False,1970-01-01 00:00:01.459793468,235.0,True,...,0,0,0,0,0,0,0,0,0,1
2,Jeff City Outsider Magazine,12,0,False,1.000000,569826598,False,1970-01-01 00:00:01.583371019,351.0,False,...,0,0,0,0,0,0,0,0,0,1
3,"Shut UP, Astoria",129,0,False,1.000000,1430964794,False,1970-01-01 00:00:01.568052024,16636.0,True,...,0,0,0,0,0,0,0,0,0,1
4,Wild Boars,41,0,False,1.326295,341849374,False,1970-01-01 00:00:01.599638720,2378.0,True,...,0,1,0,0,0,0,0,0,0,0
